In [ ]:
# Reading data from data file
import panda as pd
data_as = pd.read_csv("data_airspace.csv")

from cmath import nan
import numpy as np
import statistics

between_zero_to_five = 0
between_five_to_ten = 0
epsilon = 100

In [ ]:
# Extracting trajectory data from columns of the data file to corresponding variables

long_1 = data_as.loc[:,"longitude_1"]
lat_1 = data_as.loc[:,"latitude_1"]
alt_1 = data_as.loc[:,"altitude_1"]

long_2 = data_as.loc[:,"longitude_2"]
lat_2 = data_as.loc[:,"latitude_2"]
alt_2 = data_as.loc[:,"altitude_2"]

u_1 = data_as.loc[:,"velocity_u_1"]
v_1 = data_as.loc[:,"velocity_v_1"]
w_1 = data_as.loc[:,"velocity_w_1"]

u_2 = data_as.loc[:,"velocity_u_2"]
v_2 = data_as.loc[:,"velocity_v_2"]
w_2 = data_as.loc[:,"velocity_w_2"]


In [ ]:
# Extracting sector data from columns of the data file to corresponding variables

x_sector = data_as.loc[:,"coordinates_x"]
y_sector = data_as.loc[:,"coordinates_y"]
z_sector = data_as.loc[:,"floor_to_ceiling_height"]


In [ ]:
# 1_Number of A/C
# we should enumerate number of columns which is = (total number of columns - 3 )/6  [3 is # the columns of sector info, and 6 is 3coordinates and 3 velocities for each A/C]

No_aircraft = (len(data_as.columns) - 3)/6

In [ ]:
# 2_Area and volume of the sector
#  Shoelace formula because it is faster
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

area = PolyArea(x_sector,y_sector)
volume = area * z_sector

In [ ]:
# 3_A/C density: #A/C / Area of the sector

density = No_aircraft / volume


In [ ]:
# 4_ number of A/C with Heading Change greater than 15º
# Achtung: zero in the denominator of slope[i]
from math import pi

def Heading_change(long,lat):
    for i in range(len(long)-1):
        delta_long[i]=long[i+1]-long[i]
        delta_lat[i]=lat[i+1]-lat[i]
        slope[i]=delta_lat[i]/delta_long[i]
        arctan[i]=np.arctan( slope[i] )*180/pi
        logical_atan[i]=arctan[i]>15

    if '1' in logical_atan:
        return 1
    else:
        return 0

count_Headin_Change = Heading_change(long_1,lat_1) + Heading_change(long_2,lat_2)


In [ ]:
# 5_ number of A/C with Altitude Change greater than 750 feet (i.e., 228.6 meter)

def Altitude_Change(alt):
    delta_alt[i] = alt[i+1] - alt[i]
    logical_delta_alt[i] = delta_alt[i] > 228.6
    if '1' in logical_delta_alt:
        return 1
    else:
        return 0

count_Altitude_Change =  Altitude_Change(alt_1) + Altitude_Change(alt_2)

In [ ]:
# 6_ _ Number of intersecting flight paths i.e., number of A/C with 3D Euclidean distance lower than 'epsilon' meters
# 7_ number of A/C with 3D Euclidean distance between 0-5 nautical miles (i.e., 0-9260 meters)
# 8_ number of A/C with 3D Euclidean distance between 5-10 nautical (i.e., 9260-18520 meters)
def distance(long1,lat1,alt1, long2,lat2,alt2):

    if len(long1)>=len(long2):
        for i in range(len(long1)-1):
        diff_long[i] = long1[i]-long2[i]
        diff_lat[i] = lat1[i]-lat2[i]
        diff_alt[i] = alt1[i]-alt2[i]
        dist[i] = (diff_long[i]^2 + diff_lat[i]^2 + diff_alt[i]^2)^0.5
        logical_dist_epsilon[i] = dist[i] < epsilon
        logical_dist_5nm[i] = dist[i]<9260
        logical_dist_10nm[i] = dist[i]<18520
    else:
        for i in range(len(long2)-1):
            diff_long[i] = long1[i]-long2[i]
            diff_lat[i] = lat1[i]-lat2[i]
            diff_alt[i] = alt1[i]-alt2[i]
            dist[i] = (diff_long[i]^2 + diff_lat[i]^2 + diff_alt[i]^2)^0.5
            logical_dist_epsilon[i] = dist[i] < epsilon
            logical_dist_5nm[i] = dist[i]<9260
            logical_dist_10nm[i] = dist[i]<18520

    return sum(logical_dist_epsilon)


Number_of_intersects = distance (long_1,lat_1,alt_1, long_2,lat_2,alt_2)  
        

if '1' in logical_dist_5nm:
    between_zero_to_five = 1
elif '1' in logical_dist_10nm:
    between_five_to_ten = 1

In [ ]:
# 9_ number of A/C with lateral distance between 40-70 nautical miles and vertical separation less than 2000/1000 feet above/below 29000 ft

logical_diff_lat_1= abs(diff_lat)>40;
logical_diff_lat_2= abs(diff_lat)>70;
logical_diff_lat=logical_diff_lat_1.*logical_diff_lat_2;

logical_diff_alt_2000=diff_alt<2000;
logical_diff_alt_1000=diff_alt<1000;
logical_alt_below_2900=alt<29000;
logical_alt_above_2900=alt>29000;

Logical_diff2000_and_above=logical_diff_alt_2000.*logical_alt_above_2900;
Logical_diff1000_and_below=logical_diff_alt_1000.*logical_alt_below_2900;

Logical_overall = np.multiply (logical_diff_lat , (Logical_diff2000_and_above + Logical_diff1000_and_below))

counter=0;
if sum(Logical_overall)>0
             counter=counter+1;
end


logical_dist_1 = dist>9260
logical_dist_2 = dist<18520
logical_dist = np.multiply(logical_dist_1,logical_dist_2)

if '1' in logical_dist:
    return 1
else:
    return 0

counter=

In [ ]:
# 10_ mean of all aircrafts' altitudes in the sector
def ave_alt(alt1,alt2):
    ave1 = statistics.mean(alt1)
    ave2 = statistics.mean(alt2)
    return statistics.mean(ave1,ave2)

In [ ]:
# 11_ mean of variance of all aircrafts' altitudes in the sector
def var_alt(alt1,alt2):
    var1 = statistics.variance(alt1)
    var2 = statistics.variance(alt2)
    return statistics.mean(var1,var2)

In [ ]:
# Task load number 1: average_transit_time

def ave_tt(long):
    ave_transit_time=len(long)
    return ave_transit_time



In [ ]:
# 2_Total controller's workload
# the following parameters are borrowed from the following paper https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.207.6706&rep=rep1&type=pdf
tau_b= 2
lambda_b= 0.05
tau_t= 10
tau_r= 3
tau_c= 50
P= 300
M_h= 7*1852
M_v= 1000*0.3048
E_v12= 440*0.514444

G = tau_b*lambda_b + tau_t*density*volume/ave_tt + tau_r*density*volume/P + tau_c*2*M_h*M_v*E_v12*density*(density*volume+1)

